# Francisco Teixeira Rocha Aragão 2021031726

Nesse arquivo contem a implementação da solução para um problema de 'pos tagging', que é a tarefa de atribuir uma tag a cada palavra de uma sentença, em que no caso deste trabalho, envolve a atribuição de tags de classes gramáticais para frases em português.

Os dados utilizados são do corpus MacMorpho, que contém textos em português com as respectivas tags de classes gramaticais. O corpus já está dividido em 3 partes: treino, validação e teste. É possível baixar o corpus no [link referenciado](http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz)

Desse modo, o trabalho desenvolvido envolve a utilização de modelos para realizar a tarefa de pos tagging, com o desempenho sendo medido e retornando em cada caso testado. Vale destacar inicialmente que algumas estratégias foram testadas, porém não foram bem sucessidas, como o treinamento de muitas camadas do modelo BERT em português, ou a utilização de LLMs para a tarefa de pos tagging. A falta de sucesso deve-se a necessidade de maior poder de processamento, GPU e de memória para trabalhar com tarefas tão complexas e modelos tão grandes, o que não foi possível de ser feito localmente.

Com isso, a estratégia adotada foi de utilizar modelos já especializados na tarefa de pós taggins (fine tuning). Assim, os modelos escolhidos foram treinandos utilizando o próprio dataset MacMorpho, além de terem como base modelos BERT treinados em língua portuguesa. Fazer o uso desses modelos foi de grande ajuda pois não foi necessário o treinamento, apenas carregar em memória e utilizar.

Os resultados da utilização dos modelos estão descritos abaixo.

In [2]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import json
import os
from collections import defaultdict


/home/francisco/Documents/Francisco/Faculdade/7_Semestre/processamento_lingua_natural/tp2/pos_tagging/env_py/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def measure_and_print_general_accuracy(results, total_words):
    correct_predictions = 0
    for result in results:
        if result['correct']:
            correct_predictions += 1

    accuracy = correct_predictions / total_words
    print(f'Acurácia geral: {accuracy:.2f}')

In [4]:
def measure_and_print_accuracy_and_len_by_tag(results):
    correct_tags = {}
    total_tags = {}

    for result in results:
        if result['tag'] not in total_tags:
            total_tags[result['tag']] = 1
        else:
            total_tags[result['tag']] += 1
        
        if result['correct']:
            if result['tag'] not in correct_tags:
                correct_tags[result['tag']] = 1
            else:
                correct_tags[result['tag']] += 1

    accuracy_tags = {}
    for tag in total_tags:
        try:
            accuracy_tags[tag] = correct_tags[tag] / total_tags[tag]
        except KeyError:
            accuracy_tags[tag] = 0

    couting_tags = {}
    for tag, total in total_tags.items():
        couting_tags[tag] = total

    # print in descending order
    for tag, accuracy in sorted(accuracy_tags.items(), key=lambda x: x[1], reverse=True):
        print(f'TAG: {tag} - ACURÁCIA: {accuracy:.2f} - OCORRÊNCIAS: {couting_tags[tag]}')
    
    return total_tags, accuracy_tags


In [5]:
def measure_and_print_precision_recall_f1_by_tag(results):
    
    # contando TRUE_POSITIVES, FALSE_POSITIVES e FALSE_NEGATIVES para cada tag
    # FALSE_POSITIVE: previu como essa tag, mas não era
    # FALSE_NEGATIVE: era essa tag, mas previu como outra
    # TRUE_POSITIVE: previu como essa tag e era essa tag
    true_positives = defaultdict(int)
    false_positives = defaultdict(int)
    false_negatives = defaultdict(int)

    for result in results:
        predicted_tag = result['prediction']
        expected_tag = result['tag']
        correct_prediction = result["correct"]
        
        if correct_prediction:  # True Positive
            true_positives[predicted_tag] += 1
        else:
            # False Positive
            false_positives[predicted_tag] += 1
            # False Negative
            false_negatives[expected_tag] += 1

    # calculando precisão, recall e f1-score para cada tag
    precision = {}
    recall = {}
    f1_score = {}

    for tag in set(true_positives.keys()).union(false_positives.keys()).union(false_negatives.keys()):
        tp = true_positives[tag]
        fp = false_positives[tag]
        fn = false_negatives[tag]
        
        # Precisão: TP / (TP + FP)
        precision[tag] = tp / (tp + fp) if (tp + fp) > 0 else 0
        
        # recall: TP / (TP + FN)
        recall[tag] = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        # f1-Score: 2 * (Precision * Recall) / (Precision + Recall)
        if precision[tag] + recall[tag] > 0:
            f1_score[tag] = 2 * (precision[tag] * recall[tag]) / (precision[tag] + recall[tag])
        else:
            f1_score[tag] = 0

    for tag in sorted(f1_score.keys(), key=lambda t: f1_score[t], reverse=True):
        print(f'TAG {tag} - Precision {precision[tag]:.2f} - Recall {recall[tag]:.2f} - F1 {f1_score[tag]:.2f}')

### Primeiro modelo: Fine-tuning do modelo BioBERTpt(all) com corpus MacMorpho

In [6]:

tokenizer = AutoTokenizer.from_pretrained("pucpr-br/postagger-bio-portuguese")

model = AutoModelForTokenClassification.from_pretrained("pucpr-br/postagger-bio-portuguese")

nlp_token_class = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)




Device set to use cpu
/home/francisco/Documents/Francisco/Faculdade/7_Semestre/processamento_lingua_natural/tp2/pos_tagging/env_py/lib/python3.12/site-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [7]:
with open('data/macmorpho-train.txt', 'r') as file:
    # store file content in a list
    lines = file.readlines()
    

In [8]:
print(lines[0:4])

['Jersei_N atinge_V média_N de_PREP Cr$_CUR 1,4_NUM milhão_N na_PREP+ART venda_N da_PREP+ART Pinhal_NPROP em_PREP São_NPROP Paulo_NPROP ._PU\n', 'Programe_V sua_PROADJ viagem_N à_PREP+ART Exposição_NPROP Nacional_NPROP do_NPROP Zebu_NPROP ,_PU que_PRO-KS começa_V dia_N 25_N ._PU\n', 'Safra_N recorde_ADJ e_KC disponibilidade_N de_PREP crédito_N ativam_V vendas_N de_PREP máquinas_N agrícolas_ADJ ._PU\n', 'A_ART desertificação_N tornou_V crítica_ADJ a_ART produtividade_N de_PREP 52_NUM mil_NUM km²_N na_PREP+ART região_N ._PU\n']


In [15]:
# organizando os dados, separando as palavras das tags
words = []
tags = []
for line in lines[0:int(len(lines)*0.7)]:
    # separate each word
    words_tags = line.split()
    for word_tag in words_tags:
        # separate the word from the tag
        word, tag = word_tag.split('_')
        words.append(word)
        tags.append(tag.strip())

In [21]:
print(words[0:4])
print(tags[0:4])

['Salto', 'sete', 'O', 'grande']
['N', 'ADJ', 'ART', 'ADJ']


In [16]:
# realizando a inferência das tags
total_words = len(words)
results = []

In [17]:




for word, tag in tqdm(zip(words, tags), total=total_words, desc="Processing words"):
    prediction = nlp_token_class(word)
    result = {
        'word': word,
        'tag': tag,
        'prediction': prediction[0]['entity_group']
    }
    if prediction[0]['entity_group'] == tag:
        result['correct'] = True
    else:
        result['correct'] = False
    
    results.append(result)

# salvando resultados em um arquivo para análise posterior

with open('results/results_postagger-bio-portuguese.json', 'w') as file:
    json.dump(results, file, indent=4)

Processing words: 100%|██████████| 518670/518670 [2:17:52<00:00, 62.70it/s]  


In [18]:
# abrino o arquivo gerado
with open('results/results_postagger-bio-portuguese.json', 'r') as file:
    results = json.load(file)

In [19]:
# calculando acurácia do modelo
measure_and_print_general_accuracy(results, total_words)


Acurácia geral: 0.68


In [20]:
# observando resultados mais detalhados

# agora é calculado a acurácia para cada tag
total_tags, accuracy_tags = measure_and_print_accuracy_and_len_by_tag(results)

TAG: PREP+PROPESS - ACURÁCIA: 0.98 - OCORRÊNCIAS: 225
TAG: N - ACURÁCIA: 0.96 - OCORRÊNCIAS: 114296
TAG: KC - ACURÁCIA: 0.96 - OCORRÊNCIAS: 11980
TAG: PU - ACURÁCIA: 0.95 - OCORRÊNCIAS: 73027
TAG: V - ACURÁCIA: 0.92 - OCORRÊNCIAS: 54271
TAG: ADV - ACURÁCIA: 0.92 - OCORRÊNCIAS: 11689
TAG: PREP+ART - ACURÁCIA: 0.89 - OCORRÊNCIAS: 33970
TAG: PREP+ADV - ACURÁCIA: 0.88 - OCORRÊNCIAS: 25
TAG: PCP - ACURÁCIA: 0.79 - OCORRÊNCIAS: 11310
TAG: PROSUB - ACURÁCIA: 0.59 - OCORRÊNCIAS: 2745
TAG: PDEN - ACURÁCIA: 0.57 - OCORRÊNCIAS: 2933
TAG: NPROP - ACURÁCIA: 0.50 - OCORRÊNCIAS: 50000
TAG: PROPESS - ACURÁCIA: 0.50 - OCORRÊNCIAS: 4728
TAG: PREP+PROSUB - ACURÁCIA: 0.45 - OCORRÊNCIAS: 317
TAG: PROADJ - ACURÁCIA: 0.43 - OCORRÊNCIAS: 7102
TAG: PREP+PROADJ - ACURÁCIA: 0.37 - OCORRÊNCIAS: 889
TAG: ADJ - ACURÁCIA: 0.36 - OCORRÊNCIAS: 22321
TAG: IN - ACURÁCIA: 0.25 - OCORRÊNCIAS: 75
TAG: ART - ACURÁCIA: 0.22 - OCORRÊNCIAS: 39167
TAG: PREP - ACURÁCIA: 0.22 - OCORRÊNCIAS: 52318
TAG: KS - ACURÁCIA: 0.09 - OCORRÊ

In [21]:
measure_and_print_precision_recall_f1_by_tag(results)

TAG PREP+PROPESS - Precision 1.00 - Recall 0.98 - F1 0.99
TAG KC - Precision 0.97 - Recall 0.96 - F1 0.97
TAG PU - Precision 0.96 - Recall 0.95 - F1 0.95
TAG V - Precision 0.98 - Recall 0.92 - F1 0.95
TAG PREP+ART - Precision 0.94 - Recall 0.89 - F1 0.91
TAG PCP - Precision 0.95 - Recall 0.79 - F1 0.86
TAG PREP+ADV - Precision 0.76 - Recall 0.88 - F1 0.81
TAG N - Precision 0.67 - Recall 0.96 - F1 0.79
TAG ADV - Precision 0.63 - Recall 0.92 - F1 0.75
TAG PDEN - Precision 0.92 - Recall 0.57 - F1 0.71
TAG NPROP - Precision 0.91 - Recall 0.50 - F1 0.64
TAG PROPESS - Precision 0.75 - Recall 0.50 - F1 0.60
TAG PROADJ - Precision 0.90 - Recall 0.43 - F1 0.58
TAG PREP+PROADJ - Precision 1.00 - Recall 0.37 - F1 0.54
TAG ADJ - Precision 0.92 - Recall 0.36 - F1 0.52
TAG ART - Precision 0.96 - Recall 0.22 - F1 0.36
TAG PREP - Precision 0.94 - Recall 0.22 - F1 0.35
TAG PREP+PROSUB - Precision 0.18 - Recall 0.45 - F1 0.26
TAG IN - Precision 0.12 - Recall 0.25 - F1 0.16
TAG PROSUB - Precision 0.08 - 

### Segundo modelo: Fine-tuning do modelo BERTimbau com corpus MacMorpho

In [19]:
# carregando o modelo
pipe = pipeline("token-classification", model="lisaterumi/postagger-portuguese", tokenizer="lisaterumi/postagger-portuguese", aggregation_strategy="simple")

Device set to use cpu


In [11]:
# lendo dados de entrada novamente e separado melhor o arquivo
with open('data/macmorpho-train.txt', 'r') as file:
    # store file content in a list
    lines = file.readlines()

dataT = []

for line in lines[0:int(len(lines)*0.7)]:
    words_tags = line.split()
    for word_tag in words_tags:
        word, tag = word_tag.split('_')
        dataT.append((word, tag.strip()))


In [3]:
dataT[0:5]

[('Jersei', 'N'),
 ('atinge', 'V'),
 ('média', 'N'),
 ('de', 'PREP'),
 ('Cr$', 'CUR')]

In [4]:
# pegando todas as tags
tags = set([tag for _, tag in dataT])
tags

{'ADJ',
 'ADV',
 'ADV-KS',
 'ART',
 'CUR',
 'IN',
 'KC',
 'KS',
 'N',
 'NPROP',
 'NUM',
 'PCP',
 'PDEN',
 'PREP',
 'PREP+ADV',
 'PREP+ART',
 'PREP+PRO-KS',
 'PREP+PROADJ',
 'PREP+PROPESS',
 'PREP+PROSUB',
 'PRO-KS',
 'PROADJ',
 'PROPESS',
 'PROSUB',
 'PU',
 'V'}

In [36]:

# raelizando predição das tags
results = []

for word, expected_tag in tqdm(dataT, desc="Processing words"):
    prediction = pipe(word)
    
    # pegando a tag prevista
    predicted_tag = prediction[0]['entity_group']
    
    result_data = {
        'word': word,
        'tag': expected_tag,
        'prediction': predicted_tag,
        'correct': predicted_tag == expected_tag,
    }
    
    # Append result to the results list
    results.append(result_data)
        
# salvando resultados em um arquivo json
with open('results/results_postagger-portuguese.json', 'w') as file:
    json.dump(results, file, indent=4)
        

Processing words: 100%|██████████| 387877/387877 [1:34:09<00:00, 68.66it/s]


In [22]:
# abrindo arquivo para analise dos resultados -> acuracia geral

with open('results/results_postagger-portuguese.json', 'r') as file:
    results = json.load(file)

In [23]:

total_words = len(results)

measure_and_print_general_accuracy(results, total_words)

Acurácia geral: 0.74


In [24]:
total_tags, accuracy_tags = measure_and_print_accuracy_and_len_by_tag(results)

TAG: PU - ACURÁCIA: 1.00 - OCORRÊNCIAS: 54718
TAG: PREP+PROPESS - ACURÁCIA: 0.98 - OCORRÊNCIAS: 164
TAG: N - ACURÁCIA: 0.97 - OCORRÊNCIAS: 86505
TAG: KC - ACURÁCIA: 0.96 - OCORRÊNCIAS: 9046
TAG: ADV - ACURÁCIA: 0.92 - OCORRÊNCIAS: 8004
TAG: V - ACURÁCIA: 0.92 - OCORRÊNCIAS: 40086
TAG: PREP+ADV - ACURÁCIA: 0.89 - OCORRÊNCIAS: 19
TAG: PROSUB - ACURÁCIA: 0.88 - OCORRÊNCIAS: 1887
TAG: PREP - ACURÁCIA: 0.85 - OCORRÊNCIAS: 39865
TAG: PCP - ACURÁCIA: 0.82 - OCORRÊNCIAS: 8743
TAG: PROPESS - ACURÁCIA: 0.79 - OCORRÊNCIAS: 3366
TAG: PREP+ART - ACURÁCIA: 0.75 - OCORRÊNCIAS: 24920
TAG: PDEN - ACURÁCIA: 0.56 - OCORRÊNCIAS: 2116
TAG: ADJ - ACURÁCIA: 0.50 - OCORRÊNCIAS: 16352
TAG: NPROP - ACURÁCIA: 0.49 - OCORRÊNCIAS: 37713
TAG: PREP+PROADJ - ACURÁCIA: 0.47 - OCORRÊNCIAS: 647
TAG: PREP+PROSUB - ACURÁCIA: 0.41 - OCORRÊNCIAS: 217
TAG: IN - ACURÁCIA: 0.40 - OCORRÊNCIAS: 20
TAG: NUM - ACURÁCIA: 0.12 - OCORRÊNCIAS: 8412
TAG: ART - ACURÁCIA: 0.08 - OCORRÊNCIAS: 29287
TAG: PROADJ - ACURÁCIA: 0.05 - OCORRÊNCI

In [25]:
measure_and_print_precision_recall_f1_by_tag(results)

TAG PREP+PROPESS - Precision 1.00 - Recall 0.98 - F1 0.99
TAG KC - Precision 0.96 - Recall 0.96 - F1 0.96
TAG V - Precision 0.98 - Recall 0.92 - F1 0.95
TAG PREP - Precision 0.93 - Recall 0.85 - F1 0.89
TAG PCP - Precision 0.95 - Recall 0.82 - F1 0.88
TAG PREP+ADV - Precision 0.77 - Recall 0.89 - F1 0.83
TAG PU - Precision 0.71 - Recall 1.00 - F1 0.83
TAG PREP+ART - Precision 0.92 - Recall 0.75 - F1 0.83
TAG N - Precision 0.66 - Recall 0.97 - F1 0.79
TAG ADV - Precision 0.67 - Recall 0.92 - F1 0.77
TAG PROPESS - Precision 0.72 - Recall 0.79 - F1 0.75
TAG PDEN - Precision 0.93 - Recall 0.56 - F1 0.70
TAG NPROP - Precision 0.99 - Recall 0.49 - F1 0.65
TAG ADJ - Precision 0.88 - Recall 0.50 - F1 0.64
TAG PREP+PROADJ - Precision 0.96 - Recall 0.47 - F1 0.63
TAG PREP+PROSUB - Precision 0.30 - Recall 0.41 - F1 0.34
TAG NUM - Precision 0.89 - Recall 0.12 - F1 0.21
TAG IN - Precision 0.14 - Recall 0.40 - F1 0.21
TAG ART - Precision 0.93 - Recall 0.08 - F1 0.16
TAG PROSUB - Precision 0.07 - Rec